# 2. Retropropagación en red densa
Programa el algoritmo de retropropagación usando NumPy para una tarea de clasificación binaria presuponiendo una red densa con dos capas ocultas y la función de pérdida de entropía cruzada
binaria. Describe las fórmulas y reglas de actualización de los pesos y sesgos de cada capa y entrena
y evalúa la red en algún conjunto de datos.

In [1]:
import numpy as np

In [2]:
class MLP:
    def __init__(self, input_dim, hidden_units):
        self.input_dim = input_dim
        self.hidden_units = hidden_units
    
    def _weighted_sum(self, w, a, b):
        return np.dot(w.T, a) + b
    
    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def _derivative_sigmoid(self, x):
        return np.multiply(self._sigmoid(x), (1.0 - self._sigmoid(x)))
    
    def _binary_crossEntropy(self, y, p):
        p[p == 0] = np.nextafter(0., 1.)
        p[p == 1] = np.nextafter(1., 0.)
        return -(np.log(p[y == 1]).sum() + np.log(1 - p[y == 0]).sum())
    
    def _accuracy(self, y, y_predicted):
        return (y == y_predicted).mean() * 100
    
    def _forward_pass(self, x, w1, b1, w2, b2, w3, b3):
        a1 = x[:, np.newaxis]
        z2 = self._weighted_sum(w1, a1, b1)
        a2 = self._sigmoid(z2)
        z3 = self._weighted_sum(w2, a2, b2)
        a3 = self._sigmoid(z3)
        z4 = self._weighted_sum(w3, a3, b3)
        y_hat = self._sigmoid(z4)
        return z2, a2, z3, a3, z4, y_hat
    
    def _backpropagation(self, x, y, learning_rate, epochs):
        # Capa oculta 1 #
        w1 = np.sqrt(1.0 / self.input_dim) * np.random.randn(self.input_dim, self.hidden_units)
        b1 = np.zeros((self.hidden_units, 1))
        
        # Capa oculta 2 #
        w2 = np.sqrt(1.0 / self.hidden_units) * np.random.randn(self.hidden_units, self.hidden_units)
        b2 = np.zeros((self.hidden_units, 1))
        
        # Capa de salida #
        w3 = np.sqrt(1.0 / self.hidden_units) * np.random.randn(self.hidden_units, 1)
        b3 = np.zeros((1, 1))
        
        losses = np.zeros((epochs))
        accuracies = np.zeros((epochs))
        y_predicted = np.zeros((y.shape))
        
        for i in range(epochs):
            for j in range(x.shape[0]):
                z2, a2, z3, a3, z4, y_hat = self._forward_pass(x[j], w1, b1, w2, b2, w3, b3)
                # Gradientes para w3 y b3 por retropropagación #
                dz4 = y_hat - y[j]
                dw3 = np.outer(a3, dz4)
                db3 = dz4
                
                # Gradiente para w2 y b2 por retropropagación #
                dz3 = np.dot(w3, dz4) * self._derivative_sigmoid(z3)
                dw2 = np.outer(a2, dz3)
                db2 = dz3
                
                # Gradiente para w1 y b1 por retropropagación #
                dz2 = np.dot(w2, dz3) * self._derivative_sigmoid(z2)
                dw1 = np.outer(x[j], dz2)
                db1 = dz2
                
                # Actualización de parámetros #
                w3 = w3 - learning_rate * dw3
                b3 = b3 - learning_rate * db3
                
                w2 = w2 - learning_rate * dw2
                b2 = b2 - learning_rate * db2
                
                w1 = w1 - learning_rate * dw1
                b1 = b1 - learning_rate * db1
                
                y_predicted[j] = y_hat
            
            losses[i] = self._binary_crossEntropy(y, y_predicted)
            accuracies[i] = self._accuracy(y, np.round(y_predicted))
            print('Epoch {}: \tLoss = {} \tAccuracy = {}'.format(i, losses[i], accuracies[i]))
        return w1, b1, w2, b2, w3, b3
        
    def fit(self, x, y, learning_rate=0.01, epochs=200):
        self.w1, self.b1, self.w2, self.b2, self.w3, self.b3 = self._backpropagation(x, y, learning_rate, epochs)
    
    def predict(self, x):
        y_pred = []
        for i in range(x.shape[0]):
            _, _, _, _, _, y_hat = self._forward_pass(x[i], self.w1, self.b1, 
                                                           self.w2, self.b2, 
                                                           self.w3, self.b3)
            y_pred.append(np.round(y_hat[0]))
        return np.array(y_pred)
        
        
    

In [3]:
# ejemplo (XOR)
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0, 1, 1, 0]]).T

In [4]:
mlp = MLP(input_dim=2, hidden_units=10)

In [5]:
np.random.seed(0)
mlp.fit(X,y, learning_rate=1.0, epochs=1000)

Epoch 0: 	Loss = 5.345282830198256 	Accuracy = 25.0
Epoch 1: 	Loss = 4.281035999557765 	Accuracy = 50.0
Epoch 2: 	Loss = 3.9378163327092115 	Accuracy = 50.0
Epoch 3: 	Loss = 3.7633275438274834 	Accuracy = 50.0
Epoch 4: 	Loss = 3.6637968016068503 	Accuracy = 50.0
Epoch 5: 	Loss = 3.6003223193739697 	Accuracy = 50.0
Epoch 6: 	Loss = 3.5565379934092407 	Accuracy = 50.0
Epoch 7: 	Loss = 3.524561066858062 	Accuracy = 50.0
Epoch 8: 	Loss = 3.500182476231129 	Accuracy = 50.0
Epoch 9: 	Loss = 3.4809690057789995 	Accuracy = 50.0
Epoch 10: 	Loss = 3.4654225487968295 	Accuracy = 50.0
Epoch 11: 	Loss = 3.4525724705089464 	Accuracy = 50.0
Epoch 12: 	Loss = 3.44176295865011 	Accuracy = 50.0
Epoch 13: 	Loss = 3.4325351564515616 	Accuracy = 50.0
Epoch 14: 	Loss = 3.424558411463158 	Accuracy = 50.0
Epoch 15: 	Loss = 3.4175883932017785 	Accuracy = 50.0
Epoch 16: 	Loss = 3.411440608383793 	Accuracy = 50.0
Epoch 17: 	Loss = 3.405973102036803 	Accuracy = 50.0
Epoch 18: 	Loss = 3.401074834427126 	Accuracy =

Epoch 451: 	Loss = 2.3103833920648826 	Accuracy = 75.0
Epoch 452: 	Loss = 2.3103132470395504 	Accuracy = 75.0
Epoch 453: 	Loss = 2.3102436438453413 	Accuracy = 75.0
Epoch 454: 	Loss = 2.3101745759526082 	Accuracy = 75.0
Epoch 455: 	Loss = 2.3101060369259203 	Accuracy = 75.0
Epoch 456: 	Loss = 2.3100380204222803 	Accuracy = 75.0
Epoch 457: 	Loss = 2.309970520189391 	Accuracy = 75.0
Epoch 458: 	Loss = 2.3099035300639503 	Accuracy = 75.0
Epoch 459: 	Loss = 2.309837043969983 	Accuracy = 75.0
Epoch 460: 	Loss = 2.3097710559172095 	Accuracy = 75.0
Epoch 461: 	Loss = 2.3097055599994434 	Accuracy = 75.0
Epoch 462: 	Loss = 2.3096405503930284 	Accuracy = 75.0
Epoch 463: 	Loss = 2.3095760213552996 	Accuracy = 75.0
Epoch 464: 	Loss = 2.3095119672230853 	Accuracy = 75.0
Epoch 465: 	Loss = 2.309448382411224 	Accuracy = 75.0
Epoch 466: 	Loss = 2.3093852614111308 	Accuracy = 75.0
Epoch 467: 	Loss = 2.309322598789376 	Accuracy = 75.0
Epoch 468: 	Loss = 2.3092603891862993 	Accuracy = 75.0
Epoch 469: 	Lo

Epoch 838: 	Loss = 0.44178687778529724 	Accuracy = 100.0
Epoch 839: 	Loss = 0.40964107215118395 	Accuracy = 100.0
Epoch 840: 	Loss = 0.38179473140971054 	Accuracy = 100.0
Epoch 841: 	Loss = 0.3574440398537393 	Accuracy = 100.0
Epoch 842: 	Loss = 0.33597512418568265 	Accuracy = 100.0
Epoch 843: 	Loss = 0.3169104811953528 	Accuracy = 100.0
Epoch 844: 	Loss = 0.29987272953559796 	Accuracy = 100.0
Epoch 845: 	Loss = 0.28455946514352526 	Accuracy = 100.0
Epoch 846: 	Loss = 0.27072541137212264 	Accuracy = 100.0
Epoch 847: 	Loss = 0.25816948357307945 	Accuracy = 100.0
Epoch 848: 	Loss = 0.24672524282241254 	Accuracy = 100.0
Epoch 849: 	Loss = 0.23625373657462329 	Accuracy = 100.0
Epoch 850: 	Loss = 0.22663805220971742 	Accuracy = 100.0
Epoch 851: 	Loss = 0.21777912046756293 	Accuracy = 100.0
Epoch 852: 	Loss = 0.2095924446223505 	Accuracy = 100.0
Epoch 853: 	Loss = 0.20200552456047663 	Accuracy = 100.0
Epoch 854: 	Loss = 0.19495580883424507 	Accuracy = 100.0
Epoch 855: 	Loss = 0.1883890522958

In [6]:
y_pred = mlp.predict(X)

In [11]:
y_pred

array([[0.],
       [1.],
       [1.],
       [0.]])